<a href="https://colab.research.google.com/github/chencindyj/2017-02-20-TPL/blob/2017.02/Shopify_Intern_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Shopify Data Science Intern Challenge
###Cindy Chen

Please complete the following questions, and provide your thought process/work. You can attach your work in a text file, link, etc. on the application page. Please ensure answers are easily visible for reviewers!

###***Question 1: Given some sample data, write a program to answer the following: click here to access the required data set***

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 


a.	Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 


In [3]:
import pandas as pd
import numpy as np

In [8]:
#load data
sneaker_data = pd.read_csv("2019 Winter Data Science Intern Challenge Data Set.csv")
#preview first 10 rows of data
sneaker_data.head(10)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11
5,6,58,882,138,1,credit_card,2017-03-14 15:25:01
6,7,87,915,149,1,cash,2017-03-01 21:37:57
7,8,22,761,292,2,cash,2017-03-08 2:05:38
8,9,64,914,266,2,debit,2017-03-17 20:56:50
9,10,52,788,146,1,credit_card,2017-03-30 21:08:26


In [37]:
#calculate the initial AOV ($3145) to ensure we understand how the team initially arrived at this number
print("Average order value: $ {:.2f}".format(sneaker_data['order_amount'].mean()))

Average order value: $ 3145.13


In [11]:
#Pull descriptive statistics of the order amounts and quantities to understand outliers
sneaker_data.describe()

#These descriptive statistics highlight the presence of outliers, since the max order_amount is $704,000, but the first and third quartiles
#only range between $163 and $390.
#We can also see that the total number of items also contain a wide range due to outliers (there is a skew).

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


In [24]:
#excludethe maximum order amount to see if that yields more realistic average order values
revised_sneaker_data = sneaker_data[sneaker_data['order_amount'] != 704000]
revised_sneaker_data.describe() #not quite!

,order_id,shop_id,user_id,order_amount,total_items
count,4983.000000,4983.000000,4983.000000,4983.000000,4983.00000
mean,2501.060405,50.106362,849.918322,754.091913,1.99398
std,1443.090253,29.051718,86.800308,5314.092293,0.98318
min,1.000000,1.000000,700.000000,90.000000,1.00000
25%,1250.500000,24.000000,776.000000,163.000000,1.00000
50%,2502.000000,50.000000,850.000000,284.000000,2.00000
75%,3750.500000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,154350.000000,8.00000


In [27]:
#with the remaining data (after we removed the $704,000 outlier entries), we sort the data set by order amount to understand
#the remaining entries that are outliers.  Interestingly, we can see that they all pertain to shop_id 78.
revised_sneaker_data.sort_values(by=['order_amount'], ascending = False).head(10)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
691,692,78,878,154350,6,debit,2017-03-27 22:51:43
2492,2493,78,834,102900,4,debit,2017-03-04 4:37:34
3724,3725,78,766,77175,3,credit_card,2017-03-16 14:13:26
1259,1260,78,775,77175,3,credit_card,2017-03-27 9:27:20
4420,4421,78,969,77175,3,debit,2017-03-09 15:21:35
2564,2565,78,915,77175,3,debit,2017-03-25 1:19:35
2906,2907,78,817,77175,3,debit,2017-03-16 3:45:46
4715,4716,78,818,77175,3,debit,2017-03-05 5:10:44
3403,3404,78,928,77175,3,debit,2017-03-16 9:45:05
4192,4193,78,787,77175,3,credit_card,2017-03-18 9:25:32


In [28]:
#let's run descriptive statistics on shop 78 to understand if only a portion of their sales are absurdly high or if this is
#consistent across all their sales
shop_78 = revised_sneaker_data[revised_sneaker_data['shop_id'] == 78]
shop_78.describe()

#the descriptive statistics tell us that all its orders are unusually high. It seems like they might be a wholesaler that records its items
#in bulk units, rather than individual sneaker pairs. I make this assumption based on the total_items that they tend to sell
#(very low quantities of less than 10) and the extremely high order amounts (a mean of $49,213).

,order_id,shop_id,user_id,order_amount,total_items
count,46.000000,46.0,46.000000,46.000000,46.000000
mean,2663.021739,78.0,867.739130,49213.043478,1.913043
std,1338.520020,0.0,81.314871,26472.227449,1.029047
min,161.000000,78.0,707.000000,25725.000000,1.000000
25%,1428.250000,78.0,812.500000,25725.000000,1.000000
50%,2796.500000,78.0,866.500000,51450.000000,2.000000
75%,3720.250000,78.0,935.750000,51450.000000,2.000000
max,4919.000000,78.0,997.000000,154350.000000,6.000000


In [29]:
revised_sneaker_data2 = revised_sneaker_data[revised_sneaker_data['shop_id'] != 78]
revised_sneaker_data2.describe() #order amount looks much more intuitive!

,order_id,shop_id,user_id,order_amount,total_items
count,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000
mean,2499.551347,49.846465,849.752279,302.580514,1.994734
std,1444.069407,29.061131,86.840313,160.804912,0.982821
min,1.000000,1.000000,700.000000,90.000000,1.000000
25%,1248.000000,24.000000,775.000000,163.000000,1.000000
50%,2497.000000,50.000000,850.000000,284.000000,2.000000
75%,3751.000000,74.000000,925.000000,387.000000,3.000000
max,5000.000000,100.000000,999.000000,1760.000000,8.000000


**ANSWER:**

Using average order value (AOV) without considering outliers and the quantity of shoes per order has yielded a very high and unintuitive number.  Accordingly, excluding certain outliers based on some criteria (such as removing the maximum value or setting a threshold like anything above the third quartile) would help adjust the AOV.  Likewise, we could also report on the median order value, which is more robust against outliers and helps us understand what order value lies at the center of the order value range.

In this case, excluding the highest AOV of **704,000** as well as Store 78 would greatly improve our AOV to an amount that is more intuitive.  Instead, the average order amount would be **302.58**, which is much smaller than the initial AOV of 3145.53.

If we take the median, the median order amount (without excluding any data points) would **284.00**.  This is obtained from the original descriptive statistics that included all data points.

Since the prompt mentioned that the number seems high since sneakers are a relatively affordable item, it sounds like a complementary metric they might want is the average item value.  After all, if customers are frequently buying large quantities of a relatively affordable item, it makes sense that the average order amount is very high.  If the true goal of this business exercise is to understand the order value of the shoes that each of the 100 shops sell, a better way to evaluate this data would incorporate the quantity per order.

***b.	What metric would you report for this dataset?***

Instead of an average order value, I would report a median order value .

***c.	What is its value?***

**ANSWER:**
The average item value is 307.01. This is almost one-tenth of the initial AOV value and makes much more sense.  Overall, 307.01 likely represents more high end sneakers that the 100 sneaker shops are selling.

In [39]:
print("Median order amount: $ {:.2f}".format(sneaker_data['order_amount'].median()))

Median order amount: $ 284.00


### ***Question 2: For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.***

***a.	How many orders were shipped by Speedy Express in total?***

**ANSWER:** 54 orders

**CODE:**
SELECT COUNT(*) FROM Orders
WHERE ShipperID = 1;

***b.	What is the last name of the employee with the most orders?***

**ANSWER:**

The last name of the employee with the most orders is Peacock.

**CODE:**

SELECT TOP 1 Employees.LastName, COUNT(*) as order_count FROM Orders

INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID

GROUP BY Employees.LastName

ORDER BY COUNT(*) DESC;

***c.	What product was ordered the most by customers in Germany?***

**ANSWER:**

The product ordered the most by customers in Germany is Boston Crab Meat


**CODE:**

SELECT TOP 1 Products.ProductName, SUM(OrderDetails.Quantity) FROM ((Products

RIGHT JOIN OrderDetails on Products.ProductID = OrderDetails.ProductID)

INNER JOIN Orders on OrderDetails.OrderID = Orders.OrderID)

INNER JOIN Customers on Orders.CustomerID = Customers.CustomerID

WHERE Customers.Country = "Germany"

GROUP BY Products.ProductName

ORDER BY SUM(OrderDetails.Quantity) DESC;
